***CCN*** **Algorithm**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd

from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split
dataset_folder='/content/drive/MyDrive/Colab Notebooks/CK+48'
sub_folders=os.listdir(dataset_folder)

sub_folders
# Reading folder names as labels and images underneath
i=0
last=[]
images=[]
labels=[]
temp = sub_folders

# reading folders in the main dataset folder, one at a time
for sub_folder in sub_folders:
  sub_folder_index = temp.index(sub_folder)
  label = sub_folder_index

  # Define labels basis use case. We are using positive:0, negative:1, neutral:2
  # for our use case of predicting emotions of visitors entering a retail store
  if  label in [4, 6]:    # label in ['happy', 'surprise']
    new_label=0           # changed to label = positive emotion
  elif label in [0,5]:      # label in ['anger','sadness']
    new_label=1           # changed to label = negative emotion
  else:                   # label in ['contempt', 'disgust', 'fear']
    new_label=2           # changed to label = neutral emotion


  path = dataset_folder+'/'+sub_folder
  sub_folder_images= os.listdir(path)

  # reading images in the sub folder, one at a time
  for image in sub_folder_images:
    image_path = path+'/'+image
    print(image_path+"\t"+str(new_label))

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image= cv2.resize(image,(48,48))
    images.append(image)
    labels.append(new_label)
    i+=1
  last.append(i)
  images_x = np.array(images)
labels_y = np.array(labels)

# we divide image pixels by 255 to reduce computation power
images_x = images_x/255
# encoding the labels
num_of_classes = 3
labels_y_encoded = tf.keras.utils.to_categorical(labels_y,num_classes=num_of_classes)
X_train, X_test, Y_train, Y_test= train_test_split(images_x, labels_y_encoded,test_size=0.25, random_state=10)
len(X_train)
len(X_test)
input = Input(shape = (48,48,1))

conv1 = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size = (2,2)) (conv1)

conv2 = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size = (2,2)) (conv2)

conv3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size = (2,2)) (conv3)

conv4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size = (2,2)) (conv4)

flatten = Flatten()(pool4)

dense_1 = Dense(128,activation='relu')(flatten)

drop_1 = Dropout(0.2)(dense_1)

output = Dense(3,activation="sigmoid")(drop_1)
model = Model(inputs=input,outputs=output)
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=['accuracy'])
model.summary()
fle_s = './output/emotion_model.keras'
checkpointer = ModelCheckpoint(fle_s, monitor='loss', verbose=1, save_best_only=True,
                               save_weights_only=False, mode='auto', save_freq='epoch')

callback_list=[checkpointer]
save = model.fit(X_train,Y_train,batch_size=32,validation_data=(X_test,Y_test),epochs=50,callbacks=[callback_list])
# Checking the train and test loss and accuracy values from the neural network above.

train_loss = save.history['loss']
test_loss = save.history['val_loss']
train_accuracy = save.history['accuracy']
test_accuracy = save.history['val_accuracy']
# Plotting a line chart to visualize the loss and accuracy values by epochs.

fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)



MessageError: Error: credential propagation was unsuccessful

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# Load the trained CNN model
model = load_model('./output/emotion_model.keras')

# Define the emotion labels (you should adjust these based on your training data)
emotion_labels = ['Happy', 'Sad', 'Neutral']  # Modify this list based on your training labels

# Open the webcam (0 is the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read frame from the webcam
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break

    # Convert to grayscale (as the model was trained on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize the frame to 48x48 (input size expected by your model)
    resized_gray = cv2.resize(gray, (48, 48))

    # Normalize the pixel values (since the training data was normalized)
    normalized_img = resized_gray / 255.0

    # Expand dimensions to match the input shape of the model (batch size, height, width, channels)
    img_array = np.expand_dims(normalized_img, axis=-1)  # Add the channel dimension
    img_array = np.expand_dims(img_array, axis=0)  # Add the batch dimension

    # Predict emotion
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]  # Get the index of the highest probability

    # Get the corresponding label from the emotion_labels list
    predicted_emotion = emotion_labels[predicted_class]

    # Display the result on the webcam feed
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, f'Emotion: {predicted_emotion}', (50, 50), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame with the emotion label
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


**SVM** **Algorithm**

In [ ]:


import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Load dataset
dataset_folder = '/content/drive/MyDrive/Colab Notebooks/CK+48'
sub_folders = os.listdir(dataset_folder)

images = []
labels = []

for sub_folder in sub_folders:
    label = sub_folder
    path = dataset_folder + '/' + sub_folder
    sub_folder_images = os.listdir(path)

    for image in sub_folder_images:
        image_path = path + '/' + image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (48, 48))
        images.append(img)
        labels.append(label)

# Convert to numpy arrays
images_x = np.array(images) / 255.0
labels_y = np.array(labels)

# Flatten images for SVM
images_x_flat = images_x.reshape(images_x.shape[0], -1)

# Encode labels
lb = LabelBinarizer()
labels_y_encoded = lb.fit_transform(labels_y).argmax(axis=1)

# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(images_x_flat, labels_y_encoded, test_size=0.25, random_state=10)

# SVM Model
svm_model = SVC(kernel='rbf', C=1, gamma='scale')
svm_model.fit(X_train, Y_train)

# Predictions
Y_pred = svm_model.predict(X_test)

# Evaluation: Classification Report
print("Classification Report:")
print(classification_report(Y_test, Y_pred, target_names=lb.classes_))

# Accuracy Plot: Since SVM does not use epochs, we will just plot accuracy as a bar
accuracy = np.mean(Y_pred == Y_test) * 100


# Optional: If you want to visualize a training "loss curve", we can't plot one directly for SVM.
# However, if you'd like to simulate it, we can use cross-validation to get some "iterations" and accuracy.

# For demonstration, using the cross-validation scores (this isn't a loss curve, but accuracy at different splits)
from sklearn.model_selection import cross_val_score
svm_model_cv = SVC(kernel='rbf', C=1, gamma='scale')
cv_scores = cross_val_score(svm_model_cv, X_train, Y_train, cv=5)




**KNN** **Algorithm**

In [ ]:
from google.colab import drive
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

# Load dataset
dataset_folder = '/content/drive/MyDrive/Colab Notebooks/CK+48'
sub_folders = os.listdir(dataset_folder)

images = []
labels = []

for sub_folder in sub_folders:
    label = sub_folder
    path = dataset_folder + '/' + sub_folder
    sub_folder_images = os.listdir(path)

    for image in sub_folder_images:
        image_path = path + '/' + image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (48, 48))
        images.append(img)
        labels.append(label)

# Convert to numpy arrays
images_x = np.array(images) / 255.0
labels_y = np.array(labels)

# Flatten images for KNN
images_x_flat = images_x.reshape(images_x.shape[0], -1)

# Encode labels
lb = LabelBinarizer()
labels_y_encoded = lb.fit_transform(labels_y).argmax(axis=1)

# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(images_x_flat, labels_y_encoded, test_size=0.25, random_state=10)

# KNN Model
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, Y_train)

# Predictions
Y_pred = knn_model.predict(X_test)

# Evaluation: Classification Report
print("KNN Classification Report:")
print(classification_report(Y_test, Y_pred, target_names=lb.classes_))

# Accuracy Plot
accuracy = np.mean(Y_pred == Y_test) * 100


# Cross-validation to simulate "epochs" and accuracy for different splits
cv_scores = cross_val_score(knn_model, X_train, Y_train, cv=5)




In [ ]:
from google.colab import drive
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

# Load dataset
dataset_folder = '/content/drive/MyDrive/Colab Notebooks/CK+48'
sub_folders = os.listdir(dataset_folder)

images = []
labels = []

for sub_folder in sub_folders:
    label = sub_folder
    path = dataset_folder + '/' + sub_folder
    sub_folder_images = os.listdir(path)

    for image in sub_folder_images:
        image_path = path + '/' + image
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (48, 48))
        images.append(img)
        labels.append(label)

# Convert to numpy arrays
images_x = np.array(images) / 255.0
labels_y = np.array(labels)

# Flatten images for Random Forest
images_x_flat = images_x.reshape(images_x.shape[0], -1)

# Encode labels
lb = LabelBinarizer()
labels_y_encoded = lb.fit_transform(labels_y).argmax(axis=1)

# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(images_x_flat, labels_y_encoded, test_size=0.25, random_state=10)

# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=10)
rf_model.fit(X_train, Y_train)

# Predictions
Y_pred = rf_model.predict(X_test)

# Evaluation: Classification Report
print("Random Forest Classification Report:")
print(classification_report(Y_test, Y_pred, target_names=lb.classes_))

# Accuracy Plot
accuracy = np.mean(Y_pred == Y_test) * 100

# Cross-validation to simulate "epochs" and accuracy for different splits
cv_scores = cross_val_score(rf_model, X_train, Y_train, cv=5)

# Plotting Cross-Validation Scores



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import os
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D
from tensorflow.keras.layers import Input, Activation, Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import pandas as pd

from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, BatchNormalization, Flatten, Input
from sklearn.model_selection import train_test_split

# Set dataset folder path
dataset_folder = '/content/drive/MyDrive/Colab Notebooks/CK+48'
sub_folders = os.listdir(dataset_folder)

# Reading folder names as labels and images underneath
i = 0
last = []
images = []
labels = []
temp = sub_folders

# reading folders in the main dataset folder, one at a time
for sub_folder in sub_folders:
    sub_folder_index = temp.index(sub_folder)
    label = sub_folder_index

    # Define labels based on use case
    if label in [4, 6]:  # happy, surprise
        new_label = 0  # positive emotion
    elif label in [0, 5]:  # anger, sadness
        new_label = 1  # negative emotion
    else:  # contempt, disgust, fear
        new_label = 2  # neutral emotion

    path = dataset_folder + '/' + sub_folder
    sub_folder_images = os.listdir(path)

    # Reading images in the sub-folder, one at a time
    for image in sub_folder_images:
        image_path = path + '/' + image
        print(image_path + "\t" + str(new_label))

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (48, 48))
        images.append(image)
        labels.append(new_label)
        i += 1
    last.append(i)
    images_x = np.array(images)

labels_y = np.array(labels)

# Normalizing image data
images_x = images_x / 255.0

# Encoding the labels
num_of_classes = 3
labels_y_encoded = tf.keras.utils.to_categorical(labels_y, num_classes=num_of_classes)

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(images_x, labels_y_encoded, test_size=0.25, random_state=10)

# Defining the model architecture
input = Input(shape=(48, 48, 1))

conv1 = Conv2D(32, (3, 3), padding='same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), padding='same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), padding='same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), padding='same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

flatten = Flatten()(pool4)

dense_1 = Dense(128, activation='relu')(flatten)

drop_1 = Dropout(0.2)(dense_1)

output = Dense(3, activation="sigmoid")(drop_1)

# Compiling the model
model = Model(inputs=input, outputs=output)
model.compile(optimizer="adam", loss=["categorical_crossentropy"], metrics=['accuracy'])
model.summary()

# Define the callback for saving the best model
fle_s = './output/emotion_model.keras'
checkpointer = ModelCheckpoint(fle_s, monitor='loss', verbose=1, save_best_only=True,
                               save_weights_only=False, mode='auto', save_freq='epoch')

callback_list = [checkpointer]

# Train the model
save = model.fit(X_train, Y_train, batch_size=32, validation_data=(X_test, Y_test), epochs=50, callbacks=[callback_list])

# Plotting loss and accuracy
train_loss = save.history['loss']
test_loss = save.history['val_loss']
train_accuracy = save.history['accuracy']
test_accuracy = save.history['val_accuracy']

fig, ax = plt.subplots(ncols=2, figsize=(15, 7))
ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color='orangered', marker='o', markersize=5)
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)

# Generate predictions for both training and test sets
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Convert the predictions from probabilities to class labels (0, 1, or 2)
train_predictions = np.argmax(train_predictions, axis=1)
test_predictions = np.argmax(test_predictions, axis=1)

# Convert the true labels back to class labels
train_true_labels = np.argmax(Y_train, axis=1)
test_true_labels = np.argmax(Y_test, axis=1)

# Classification report for the training set
train_classification_report = classification_report(train_true_labels, train_predictions, target_names=['Positive', 'Negative', 'Neutral'])
print("Training Classification Report:\n")
print(train_classification_report)

# Classification report for the test set
test_classification_report = classification_report(test_true_labels, test_predictions, target_names=['Positive', 'Negative', 'Neutral'])
print("\nTesting Classification Report:\n")
print(test_classification_report)

# Overall classification report for the entire dataset
# Combining both training and test labels
all_true_labels = np.concatenate((train_true_labels, test_true_labels))
all_predictions = np.concatenate((train_predictions, test_predictions))

overall_classification_report = classification_report(all_true_labels, all_predictions, target_names=['Positive', 'Negative', 'Neutral'])
print("\nOverall Classification Report:\n")
print(overall_classification_report)
